In [34]:
%load_ext autoreload
%autoreload 2
from slack import SlackInteractor
from llm import LLMInteractor
from runner import process_threads

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
slack_interactor = SlackInteractor()
llm_interactor = LLMInteractor()

In [36]:
data = slack_interactor.fetch_new_messages()
threads = slack_interactor.organize_threads(data)
results = process_threads(slack_interactor, llm_interactor, threads)

# Analyze the results
for result in results:
    print(f"Thread in channel {result['channel']}:")
    print(f"  New items: {result['new_items']}")
    print(f"  Time since last activity: {result['time_since_last_activity']}")
    print(f"  Reminder sent: {result['reminder_sent']}")
    print(f"  Reminder content: {result['reminders']}")
    print("---")

Set conversations_oldest to 1722399936.894109
Loaded 50 old messages
Rate limited. Retrying in 1.99 seconds (attempt 1/5)
Rate limited. Retrying in 2.04 seconds (attempt 2/5)
Found 0 new messages
Updated complete_conversations.pkl with new messages

Reviewing thread in channel: general
Thread timestamp: 2024-08-06 17:31:16.182668800
Last message content:
can everyone give an answer please?
Debug - Total completed items: 0
Debug - Adding new item: Provide a preferred location for the next offsite (Assignee: Unassigned)
Debug - Total new items identified: 1

Raw LLM Response:
{
    "action_items": [
        {
            "description": "Provide a preferred location for the next offsite",
            "assignee": "Unassigned"
        }
    ],
    "completed_tasks": []
}

New action items identified: Provide a preferred location for the next offsite (Assignee: Unassigned)
Thread active within last 24 hours (last activity: 0 days 00:01:33.876560120 ago). No reminder sent.

Thread in channel 

In [37]:
llm_interactor.action_db.get_all_open_thread_ids()

['2024-08-06 17:31:16.182668800']

In [38]:
slack_interactor.load_old_messages('complete_conversations.pkl')

,type,subtype,ts,user,thread_ts,text,channel_id,text_clean,text_len,user_name,is_bot,channel_name
0,message,NaN,2024-08-06 22:27:06.140538931,U07EU2TQ6HG,2024-08-06 17:31:16.182668924,can everyone give an answer please?,C07EWH2GX7W,can everyone give an answer please?,35,Eli,False,general
1,message,NaN,2024-08-06 22:05:12.041878939,U07EU2TQ6HG,2024-08-06 17:31:16.182668924,can everyone give an answer please?,C07EWH2GX7W,can everyone give an answer please?,35,Eli,False,general
2,message,NaN,2024-08-06 21:28:42.285388947,U07FC50L9LZ,2024-08-02 18:09:15.892759085,Here's a friendly reminder for the open action...,C07EWH2GX7W,Here's a friendly reminder for the open action...,475,Agentflow,True,general
3,message,NaN,2024-08-06 17:44:11.076818943,U07EU2TQ6HG,2024-08-06 17:31:16.182668924,can everyone give an answer please?,C07EWH2GX7W,can everyone give an answer please?,35,Eli,False,general
4,message,NaN,2024-08-06 17:31:16.182668924,U07EU2TQ6HG,2024-08-06 17:31:16.182668924,could everyone tell me here where you want our...,C07EWH2GX7W,could everyone tell me here where you want our...,60,Eli,False,general
5,message,NaN,2024-08-06 17:17:26.753889084,U07EU2TQ6HG,NaT,could everyone tell me here where you want our...,C07EWH2GX7W,could everyone tell me here where you want our...,60,Eli,False,general
6,message,NaN,2024-08-06 17:16:43.789098978,U07EU2TQ6HG,NaT,could everyone tell me here where you want our...,C07EWH2GX7W,could everyone tell me here where you want our...,60,Eli,False,general
7,message,NaN,2024-08-06 17:10:23.496629000,U07EU2TQ6HG,NaT,could everyone tell me here where you want our...,C07EWH2GX7W,could everyone tell me here where you want our...,60,Eli,False,general
8,message,NaN,2024-08-06 17:06:41.242619038,U07EU2TQ6HG,NaT,could everyone tell me here where you want our...,C07EWH2GX7W,could everyone tell me here where you want our...,60,Eli,False,general
9,message,NaN,2024-08-06 16:56:48.744949102,U07EU2TQ6HG,NaT,could everyone tell me here where you want our...,C07EWH2GX7W,could everyone tell me here where you want our...,60,Eli,False,general
